### Тензоры, способы создания и атрибуты

Многомерный массив элементов одного типа.

Будем использовать обозначение $X \sim (s_1, s_2, ..., s_n)$ для формы тензора.

Примеры:
- $x \sim (d)$ — $d$-мерный вектор
- $X \sim (n, m)$ — матрица $n \times m$
- $X \sim (b, 3, h, w)$ — тензор, представляющий батч из b RGB-изображений размера $h \times w$

Всё очень похоже на `numpy`, поэтому будет полезно повторить:
- Индексация: https://numpy.org/doc/stable/user/basics.indexing.html
- Broadcasting (о нём будет речь ниже): https://numpy.org/doc/stable/user/basics.broadcasting.html

Про тензоры в Pytorch:
- https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html
- https://pytorch.org/tutorials/beginner/introyt/tensors_deeper_tutorial.html

Основные атрибуты: ранг (`dim`), размерности (`shape`), тип значений (`type`), место размещения (`device`)

In [1]:
import torch

print(torch.__version__)
print(f"CUDA available: {torch.cuda.is_available()}")

2.8.0
CUDA available: False


In [2]:
# из списка
x = torch.tensor([[1, 2, 3], [4, 5, 6]])
# посмотрим сам тензор и его атрибуты
print(x)
print(
    "\nRank: ", x.dim(),
    "\nShape: ", x.shape,
    "\nDevice: ", x.device,
    "\ntype: ", x.type(),
)

tensor([[1, 2, 3],
        [4, 5, 6]])

Rank:  2 
Shape:  torch.Size([2, 3]) 
Device:  cpu 
type:  torch.LongTensor


In [3]:
# примеры: сложение, умножение на скаляр, агрегация, матричное умножение
y = torch.tensor([[1, -2, 3], [-4, 5, -6]]).float()
print(x + y)
print(0.5 * x)
print(y.mean(dim=0))
print(x.float() @ y.T)

tensor([[ 2.,  0.,  6.],
        [ 0., 10.,  0.]])
tensor([[0.5000, 1.0000, 1.5000],
        [2.0000, 2.5000, 3.0000]])
tensor([-1.5000,  1.5000, -1.5000])
tensor([[  6., -12.],
        [ 12., -27.]])


Ещё пример: линейная регрессия

In [4]:
# случай с 1 наблюдением
x = torch.randn(4)
w = torch.randn(4, 5)
b = torch.randn(5)
y = x @ w + b  # тут всё ок, x @ w: [5], b: [5], формы совпадают
print("Форма тензора с предсказаниями:", y.shape)

# случай с 10 наблюдениями
X = torch.randn(10, 4)
Y = X @ w + b
# Z = X @ w: [10, 4] x [4, 5] -> [10, 5]
# Y = Z + b: [10, 5] + [5] -> [10, 5] ???
print("Форма тензора с предсказаниями:", Y.shape)

Форма тензора с предсказаниями: torch.Size([5])
Форма тензора с предсказаниями: torch.Size([10, 5])


#### 1.2. Broadcasting

https://pytorch.org/tutorials/beginner/introyt/tensors_deeper_tutorial.html#in-brief-tensor-broadcasting

Некоторые операции поддерживают `broadcast`, то есть размерности аргументов автоматически расширяются до нужного размера без копирования данных
Благодаря `broadcast` многие вещи получается описать лаконично.

<img src="../assets/images/02.05-broadcasting.png" />


Общие правила, когда это работает:
1. Все тензоры не пустые
2. При сравнении размеров тензоров, начиная с последней:
   1. Размерности совпадают, или
   2. Одна из размерностей равна $1$, или
   3. Размерность отсутствует в одном из тензоров

Возможно ли сложить тензоры `x` и `y` в примерах снизу?

In [5]:
# x=torch.empty(5,7,3)
# y=torch.empty(5,7,3)

# x=torch.empty((0,))
# y=torch.empty(2,2)

# x=torch.empty(5,3,4,1)
# y=torch.empty(  3,1,1)

# x=torch.empty(5,2,4,1)
# y=torch.empty(  3,1,1)

# (x + y).shape

Пример: посчитаем матрицу попарных разностей между элементами пары векторов

In [6]:
x = torch.arange(6)
y = torch.arange(4)
print(x)
print(y)
# преобразуем форму x к (6, 1) и попробуем вычесть y
x.view(-1, 1) - y
# формы тензоров x: (6, 1), y: (4,), последняя размерность в x равна 1,
# а следующая отсутствует в y, значит работает broadcast
# x (6, 1)
# y (   4)

tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3])


tensor([[ 0, -1, -2, -3],
        [ 1,  0, -1, -2],
        [ 2,  1,  0, -1],
        [ 3,  2,  1,  0],
        [ 4,  3,  2,  1],
        [ 5,  4,  3,  2]])